# Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/Shareddrives/Thesis (PS4)/[01] Colab/[02] MD/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/Shareddrives/Thesis (PS4)/[01] Colab/[02] MD/'
/content/gdrive/Shareddrives/Thesis (PS4)/[01] Colab/[02] MD


#Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import joblib
import os
import glob

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error, median_absolute_error, explained_variance_score, max_error
from sklearn.model_selection import ShuffleSplit

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

#Hyperparameter Optimization

In [ ]:
XYFeatureSelection = pd.read_csv ('pIC50_correlation_processed_KlekotaRothCount.csv')
Y = XYFeatureSelection['pIC50']
X = XYFeatureSelection.drop(columns=['pIC50'])

In [ ]:
# Split 3 - best split
#random state = 42 + split number
#will use 42 + 3 = 45 random state for splitting

In [ ]:
#internal external split
X_internal, X_external, Y_internal, Y_external = train_test_split(X, Y,
                                                                  test_size=0.2,
                                                                  random_state=45)

## RandomSearchCV

In [ ]:
param_dist = {
    'n_estimators': randint(100, 1000),
    'max_depth': [None] + list(randint(5, 100).rvs(50)),
    'min_samples_split': randint(2, 10),
    'max_features': ['sqrt', 'log2', 'none', 1]
}

model = RandomForestRegressor(random_state=42, n_jobs=-1, verbose=1)

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=500,
    cv=3,
    random_state=42,
    verbose = 1,
    scoring='neg_root_mean_squared_error'
)

In [ ]:
random_search_model = random_search.fit(X_internal, Y_internal)

In [ ]:
# View best hyperparameters
best_params_random = random_search.best_params_
print ("Best Hyperparameters (Random Search):", best_params_random)

Best Hyperparameters (Random Search): {'max_depth': 73, 'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 942}


In [ ]:
# run 1 Best Hyperparameters (Random Search): {'max_depth': 61, 'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 942}
# run 2 Best Hyperparameters (Random Search): {'max_depth': 47, 'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 942}
# run 3 Best Hyperparameters (Random Search): {'max_depth': 32, 'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 746}
# run 4 Best Hyperparameters (Random Search): {'max_depth': 33, 'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 942}
# rub 5 Best Hyperparameters (Random Search): {'max_depth': 73, 'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 942}


In [ ]:
# Get best estimator based on best parameters, and use it to predict on external set
best_model_random = random_search.best_estimator_
random_search_Y_external_pred = best_model_random.predict(X_external)

mae_random = mean_absolute_error(Y_external, random_search_Y_external_pred)
rmse_random = np.sqrt(mean_squared_error(Y_external, random_search_Y_external_pred))
r2_random = r2_score(Y_external, random_search_Y_external_pred)

random_search_optimized_model_metrics = pd.DataFrame({
    'Metric': ['MAE', 'RMSE', 'R2'],
    'Value': [mae_random, rmse_random, r2_random]
})
random_search_optimized_model_metrics

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 942 out of 942 | elapsed:    0.4s finished


,Metric,Value
0,MAE,0.595592
1,RMSE,0.784707
2,R2,0.688619


## Grid Search

In [ ]:
rf_param_grid = {
    'n_estimators': range(740,960,20),
    'max_depth': range(30, 80, 10),
    'min_samples_split': [4],
    'max_features': ['sqrt']
}

model = RandomForestRegressor(random_state=42, n_jobs=-1, verbose=1)
grid_search = GridSearchCV(
    estimator=model,
    param_grid=rf_param_grid,
    cv=3,
    verbose=2,
    scoring='neg_mean_squared_error'
)

In [ ]:
grid_search_model2 = grid_search.fit(X_internal, Y_internal)

Fitting 3 folds for each of 55 candidates, totalling 165 fits


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 740 out of 740 | elapsed:    7.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 740 out of 740 | elapsed:    0.4s finished


[CV] END max_depth=30, max_features=sqrt, min_samples_split=4, n_estimators=740; total time=   8.0s


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 740 out of 740 | elapsed:    4.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 740 out of 740 | elapsed:    0.3s finished


[CV] END max_depth=30, max_features=sqrt, min_samples_split=4, n_estimators=740; total time=   4.8s


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 940 out of 940 | elapsed:    7.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 940 out of 940 | elapsed:    0.4s finished


[CV] END max_depth=70, max_features=sqrt, min_samples_split=4, n_estimators=940; total time=   7.8s


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 940 out of 940 | elapsed:    5.5s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 940 out of 940 | elapsed:    0.5s finished


[CV] END max_depth=70, max_features=sqrt, min_samples_split=4, n_estimators=940; total time=   6.3s


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 940 out of 940 | elapsed:    9.0s finished


In [ ]:
best_params_grid = grid_search.best_params_
print ("Best Hyperparameters (grid Search):", best_params_grid)

Best Hyperparameters (grid Search): {'max_depth': 60, 'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 940}


In [ ]:
best_model_grid = grid_search.best_estimator_
grid_search_Y_external_pred = best_model_grid.predict(X_external)


mae_random = mean_absolute_error(Y_external, grid_search_Y_external_pred)
rmse_random = np.sqrt(mean_squared_error(Y_external, grid_search_Y_external_pred))
r2_random = r2_score(Y_external, grid_search_Y_external_pred)


grid_search_optimized_model_metrics = pd.DataFrame({
    'Metric': ['MAE', 'RMSE', 'R2'],
    'Value': [mae_random, rmse_random, r2_random]
})

grid_search_optimized_model_metrics

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 940 out of 940 | elapsed:    0.5s finished


,Metric,Value
0,MAE,0.595570
1,RMSE,0.784688
2,R2,0.688635
